Credit goes to https://colab.research.google.com/github/count0/colab-gt/blob/master/colab-gt.ipynb for the steps to use graph_tool in Colab.

In [ ]:
!echo "deb http://downloads.skewed.de/apt jammy main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard
!pip install gravis

In [8]:
from graph_tool.all import *

class MyList:
    def __init__(self, arr):
        self.arr = arr
    # https://stackoverflow.com/questions/1047021/overriding-in-python-iadd-method
    # https://stackoverflow.com/questions/5082190/typeerror-after-overriding-the-add-method
    def __add__(self,other_arr):
        if type(other_arr) == list:
            self.arr.extend(other_arr)
        if type(other_arr) == MyList:
            self.arr.extend(other_arr.arr)
        return self
    def __radd__(self,other_arr):
        if type(other_arr) == list:
            self.arr.extend(other_arr)
        if type(other_arr) == MyList:
            self.arr.extend(other_arr.arr)
        return self

gtest = Graph()
v1 = gtest.add_vertex()
v2 = gtest.add_vertex()
v3 = gtest.add_vertex()
v4 = gtest.add_vertex()
v5 = gtest.add_vertex()
gtest.add_edge(v1, v2)
gtest.add_edge(v2, v3)
gtest.add_edge(v4, v3)
gtest.add_edge(v3, v5)
gtest.add_edge(v2, v5)
vprop = gtest.new_vertex_property("object")
vprop[v1] = MyList([i for i in range(4)])
vprop[v2] = MyList([i for i in range(100, 107)])
vprop[v3] = MyList([i for i in range(10, 15)])
vprop[v4] = MyList([i for i in range(32, 37)])
vprop[v5] = MyList([i for i in range(73, 76)])
for i in vprop:
    print(i.arr)
parts = gtest.new_vertex_property("int")
parts[v1] = 0
parts[v2] = 0
parts[v3] = 1
parts[v4] = 1
parts[v5] = 1

# condg, condparts, vcount, ecount, avp, aep = condensation_graph(gtest, parts, avprops=[vprop])
# print(condparts.a)
# print(vcount.a)
# print(ecount.a)
# for i in range(condg.num_vertices()):
#     print(avp[0][i].arr)

[0, 1, 2, 3]
[100, 101, 102, 103, 104, 105, 106]
[10, 11, 12, 13, 14]
[32, 33, 34, 35, 36]
[73, 74, 75]


In [27]:
from datetime import datetime

g = load_graph("cleaned_HepPh.gt.gz")

In [ ]:
import time
from collections import Counter
from tabulate import tabulate

def getModularity(G, parts_idx, edge_weights):
    return modularity(G, parts_idx, weight=edge_weights)

def moveNodes(G, parts_idx, edge_weights):
    # https://arxiv.org/pdf/1810.08473.pdf
    count = 0
    while True:
        mod_old_old = getModularity(G, parts_idx, edge_weights)
        mod_old = mod_old_old
        for v in G.iter_vertices():
            temp = parts_idx[v]
            curr_max_mod = -10
            curr_best_swap = -1
            for s,t in G.iter_all_edges(v):
                if s == v and parts_idx[t] != temp:
                    parts_idx[s] = parts_idx[t]
                    mod_new = getModularity(G, parts_idx, edge_weights)
                    if mod_new > curr_max_mod:
                        curr_max_mod = mod_new
                        curr_best_swap = parts_idx[s]
                    parts_idx[s] = temp
                elif t == v and parts_idx[s] != temp:
                    parts_idx[t] = parts_idx[s]
                    mod_new = getModularity(G, parts_idx, edge_weights)
                    if mod_new > curr_max_mod:
                        curr_max_mod = mod_new
                        curr_best_swap = parts_idx[t]
                    parts_idx[t] = temp
            if curr_max_mod > mod_old:
                mod_old = curr_max_mod
                parts_idx[v] = curr_best_swap
            print(f"After vertex {v+1}:", mod_old)
        count += 1
        mod_new = getModularity(G, parts_idx, edge_weights)
        print(f"After moveNodes iteration {count}:", mod_new)
        if mod_new == mod_old_old:
            break
    return len(set(list(parts_idx)))

def Louvain(G):
    # https://arxiv.org/pdf/1810.08473.pdf
    G_copy = G.copy()
    master_partition = G_copy.new_vp("object")
    num_partitions = G_copy.num_vertices()
    for idx,v in enumerate(G_copy.iter_vertices()):
        master_partition[v] = MyList([idx])
    partition_idxs = G_copy.new_vp("int")
    for i in range(G_copy.num_vertices()):
        partition_idxs[i] = i
    curr_iteration = 0
    edge_weights = G_copy.new_ep("int")
    edge_weights.set_value(1)
    while True:
        old_mod = getModularity(G_copy, partition_idxs, edge_weights)
        num_partitions = moveNodes(G_copy, partition_idxs, edge_weights)
        curr_iteration += 1
        new_mod = getModularity(G_copy, partition_idxs, edge_weights)
        print(f"After iteration {curr_iteration}", new_mod)
        if abs(old_mod - new_mod) < 0.0000001:
            break
        G_copy, partition_idxs, vcount, ecount, avp, aep = condensation_graph(G_copy, partition_idxs, avprops=[master_partition], aeprops=[edge_weights], self_loops=True)
        edge_weights = aep[0]
        master_partition = avp[0]
    final_part_idxs = G.new_vp("int")
    for part_idx,part in enumerate(master_partition):
        for node in part.arr:
            final_part_idxs[node] = part_idx
    return final_part_idxs

g.set_directed(False)
print(g.is_directed())
part_idxs = Louvain(g)
with open("louvaincomms.txt", "w") as f:
    print(part_idxs, file=f)
# Use the below instead of above to test Louvain on the famous Zachary's Karate Club Dataset
# Results are validated by https://scholarworks.sjsu.edu/cgi/viewcontent.cgi?article=1528&context=etd_projects
# g = collection.data["karate"]
# part_idxs = Louvain(g)

# https://stackoverflow.com/questions/12282232/how-do-i-count-occurrence-of-unique-values-inside-a-list
parts = Counter(part_idxs).keys() # equals to list(set(part_idxs))-
freq = Counter(part_idxs).values() # counts the elements' frequency

# https://stackoverflow.com/questions/41140647/python-printing-lists-with-tabulate
table = zip(parts, freq)
print(tabulate(table, headers=["Community Index", "Number of Nodes"]))

In [31]:
with open("louvaincomms.txt", "w") as f:
    for i in part_idxs.a:
        print(i, file=f)